In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [3]:
import pandas as pd
import re

def preprocess_nutrition_data(df):
    # 1. 단위를 추출하기 위한 정규표현식 (g, mg, ug, mcg 등)
    # 컬럼명 예: "단백질(g)", "나트륨(mg)", "비타민A(ug)"
    unit_pattern = re.compile(r'\((g|mg|ug|mcg)\)', re.IGNORECASE)

    # 단위 변환 계수
    unit_map = {
        'g': 1.0,
        'mg': 0.001,
        'ug': 0.000001,
        'mcg': 0.000001
    }

    # 변환된 g 값을 저장할 임시 데이터프레임
    df_in_grams = pd.DataFrame()
    nutrient_cols = []

    for col in df.columns:
        match = unit_pattern.search(col)
        if match:
            unit = match.group(1).lower()
            nutrient_cols.append(col)
            # 해당 컬럼의 값을 g 단위로 환산
            df_in_grams[col] = pd.to_numeric(df[col], errors='coerce').fillna(0) * unit_map[unit]

    # 2. 행별 성분 총합 계산 (100g 당 함량이므로 모든 성분의 합은 100g 이하여야 함)
    df['total_grams'] = df_in_grams.sum(axis=1)

    # 3. 이상치 판단 및 삭제 (100g을 초과하는 행 제외)
    # 부동 소수점 오차를 고려하여 약간의 여유(예: 100.0001)를 둘 수 있습니다.
    cleaned_df = df[df['total_grams'] <= 100.0001].copy()

    # 결과 확인을 위해 추가했던 total_grams 컬럼 삭제 (선택 사항)
    # cleaned_df = cleaned_df.drop(columns=['total_grams'])

    print(f"전체 데이터: {len(df)}행")
    print(f"삭제된 이상치: {len(df) - len(cleaned_df)}행")

    return cleaned_df

# 함수 실행
df = pd.read_excel('/content/20251229_음식DB 19495건.xlsx')
cleaned_df = preprocess_nutrition_data(df)

전체 데이터: 19495행
삭제된 이상치: 2096행


/tmp/ipython-input-3434407710.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_in_grams[col] = pd.to_numeric(df[col], errors='coerce').fillna(0) * unit_map[unit]
/tmp/ipython-input-3434407710.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_in_grams[col] = pd.to_numeric(df[col], errors='coerce').fillna(0) * unit_map[unit]
/tmp/ipython-input-3434407710.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joi

In [4]:
print(cleaned_df.head())

                    식품코드                   식품명 데이터구분코드 데이터구분명  식품기원코드  \
482  D201-013000000-0001  리소토/리조또_스파이시 씨푸드 리조또       D     음식       2   
483  D201-013000000-0002     리소토/리조또_치킨 크림 리조또       D     음식       2   
484  D201-013000000-0003      리소토/리조또_베이컨크림리조또       D     음식       2   
485  D201-017000000-0001        볶음밥_갈비천왕 치즈 치밥       D     음식       2   
486  D201-017000000-0002          볶음밥_달걀듬뿍 볶음밥       D     음식       2   

                      식품기원명  식품대분류코드 식품대분류명  대표식품코드    대표식품명  ...  출처코드  \
482  외식(프랜차이즈 등 업체 제공 영양정보)        1     밥류    1013  리소토/리조또  ...     3   
483  외식(프랜차이즈 등 업체 제공 영양정보)        1     밥류    1013  리소토/리조또  ...     3   
484  외식(프랜차이즈 등 업체 제공 영양정보)        1     밥류    1013  리소토/리조또  ...     3   
485  외식(프랜차이즈 등 업체 제공 영양정보)        1     밥류    1017      볶음밥  ...     3   
486  외식(프랜차이즈 등 업체 제공 영양정보)        1     밥류    1017      볶음밥  ...     3   

          출처명  1인(회)분량 참고량  식품중량    업체명 데이터생성방법코드 데이터생성방법명     데이터생성일자  \
482  식품의약품안전처          NaN  241g    

In [5]:
print(cleaned_df.columns.tolist())

['식품코드', '식품명', '데이터구분코드', '데이터구분명', '식품기원코드', '식품기원명', '식품대분류코드', '식품대분류명', '대표식품코드', '대표식품명', '식품중분류코드', '식품중분류명', '식품소분류코드', '식품소분류명', '식품세분류코드', '식품세분류명', '영양성분함량기준량', '에너지(kcal)', '수분(g)', '단백질(g)', '지방(g)', '회분(g)', '탄수화물(g)', '당류(g)', '식이섬유(g)', '칼슘(mg)', '철(mg)', '인(mg)', '칼륨(mg)', '나트륨(mg)', '비타민A(μg RAE)', '레티놀(μg)', '베타카로틴(μg)', '티아민(mg)', '리보플라빈(mg)', '니아신(mg)', '비타민 C(mg)', '비타민 D(μg)', '콜레스테롤(mg)', '포화지방산(g)', '트랜스지방산(g)', '니코틴산(mg)', '니코틴아마이드(mg)', '비오틴(μg)', '비타민 B6 / 피리독신(mg)', '비타민 B12(μg)', '엽산(μg DFE)', '콜린(mg)', '판토텐산(mg)', '비타민 D2(μg)', '비타민 D3(μg)', '비타민 E(mg α-TE)', '알파 토코페롤(mg)', '베타 토코페롤(mg)', '감마 토코페롤(mg)', '델타 토코페롤(mg)', '알파 토코트리에놀(mg)', '베타 토코트리에놀(mg)', '감마 토코트리에놀(mg)', '델타 토코트리에놀(mg)', '비타민 K(μg)', '비타민 K1(μg)', '비타민 K2(μg)', '갈락토오스(g)', '과당(g)', '당알콜(g)', '맥아당(g)', '알룰로오스(g)', '에리스리톨(g)', '유당(g)', '자당(g)', '타가토스(g)', '포도당(g)', '불포화지방산(g)', 'EPA와 DHA의 합(mg)', '가돌레산(20:1) / 에이코센산(20:1)(mg)', '감마 리놀렌산(18:3(n-6))(mg)', '네르본산(24:1)(mg)', '도코사디에노산(22:2)(mg)', '

In [7]:
import pandas as pd
import re

def preprocess_nutrition_data(df):
    # 1. 수치 데이터 정제 (결측치 처리 및 숫자 변환)
    # 분석 대상이 될 수 있는 주요 컬럼 및 수치형 컬럼들을 정리
    target_cols = ['탄수화물(g)', '단백질(g)', '지방(g)', '수분(g)', '회분(g)', '에너지(kcal)']

    for col in df.columns:
        # 데이터에 섞여 있을 수 있는 문자열(예: "미량", "-", " ")을 처리
        if df[col].dtype == 'object':
            # 숫자가 아닌 값은 NaN으로 만든 뒤 0으로 채움
            df[col] = pd.to_numeric(df[col].astype(str).str.replace(' ', ''), errors='coerce').fillna(0)

        # 음수 데이터 보정 (물리적으로 불가능함)
        if df[col].dtype in ['float64', 'int64']:
            df.loc[df[col] < 0, col] = 0

    # 2. 검증에 사용할 '최상위 카테고리' 컬럼 지정
    # 이 성분들은 서로 독립적이며, 이들의 합은 100g을 넘을 수 없습니다.
    major_nutrients = ['탄수화물(g)', '단백질(g)', '지방(g)', '수분(g)', '회분(g)']

    # 데이터프레임에 실제 존재하는 컬럼만 필터링 (오류 방지)
    existing_major = [col for col in major_nutrients if col in df.columns]

    # 3. 행별 성분 총합 계산 (100g 기준 검증)
    df['total_major_grams'] = df[existing_major].sum(axis=1)

    # 4. 이상치 판단 및 삭제
    # 실험 오차 및 반올림을 고려하여 100.5g을 임계값으로 설정
    threshold = 100.5
    cleaned_df = df[df['total_major_grams'] <= threshold].copy()

    # 5. (선택) 에너지(kcal) 교차 검증 - 비정상 데이터 출력만 확인
    # 탄(4) + 단(4) + 지(9) 계산식과 실제 에너지의 차이가 너무 큰 경우
    #calculated_kcal = (df['탄수화물(g)'] * 4) + (df['단백질(g)'] * 4) + (df['지방(g)'] * 9)
    #kcal_diff_mask = (abs(df['에너지(kcal)'] - calculated_kcal) > 500) # 차이가 500kcal 이상인 경우 예시

    print(f"--- 전처리 결과 요약 ---")
    print(f"원본 데이터 행 수: {len(df)}")
    print(f"삭제된 이상치(100g 초과): {len(df) - len(cleaned_df)}행")
    print(f"남은 데이터 행 수: {len(cleaned_df)}")

    return cleaned_df

# 사용 예시:
df = pd.read_excel('/content/20251229_음식DB 19495건.xlsx')
cleaned_df = preprocess_nutrition_data(df)

--- 전처리 결과 요약 ---
원본 데이터 행 수: 19495
삭제된 이상치(100g 초과): 490행
남은 데이터 행 수: 19005


ㄴ 영양값이 100g 기준으로 작성되지 않았을 가능성 있는 행 삭제

In [9]:
import pandas as pd
import re
import numpy as np

def preprocess_nutrition_data(input_df):
    # 원본 보존을 위해 복사본 생성
    df = input_df.copy()

    # --- [1단계] 단위 추출 및 g 단위 환산 전용 임시 DF 생성 ---
    unit_pattern = re.compile(r'\((g|mg|ug|mcg)\)', re.IGNORECASE)
    unit_map = {'g': 1.0, 'mg': 0.001, 'ug': 0.000001, 'mcg': 0.000001}

    # 계산만을 위한 임시 데이터프레임 (원본 df에 컬럼이 추가되지 않음)
    temp_grams = pd.DataFrame(index=df.index)

    for col in df.columns:
        # 데이터 정제: 숫자 외 문자 제거 및 숫자 변환
        if df[col].dtype == 'object':
            df[col] = pd.to_numeric(df[col].astype(str).str.replace(' ', ''), errors='coerce').fillna(0)

        # 음수 값 0으로 보정 (원본 df 업데이트)
        if df[col].dtype in ['float64', 'int64']:
            df.loc[df[col] < 0, col] = 0

        # 단위 매칭 및 g 환산하여 임시 DF에 저장
        match = unit_pattern.search(col)
        if match:
            unit = match.group(1).lower()
            temp_grams[col] = df[col] * unit_map[unit]

    # --- [2단계] 최상위 카테고리 기반 100g 검증 ---
    major_nutrients = ['탄수화물(g)', '단백질(g)', '지방(g)', '수분(g)', '회분(g)']
    existing_major = [c for c in major_nutrients if c in temp_grams.columns]

    # 100g 당 함량 합계 계산
    df['total_major_grams'] = temp_grams[existing_major].sum(axis=1)

    # 100.5g 임계값 적용하여 필터링된 새로운 데이터프레임 생성
    # 기존 변수를 덮어쓰지 않고 'refined_df'에 저장
    refined_df = df[df['total_major_grams'] <= 100.5].copy()

    # --- [3단계] 정밀 에너지 교차 검증 (refined_df 대상) ---
    def get_v(c): return refined_df[c] if c in refined_df.columns else 0

    # 순수 탄수화물 계산 (탄수화물 총량 - 저에너지 성분들)
    net_carb = (get_v('탄수화물(g)') - get_v('식이섬유(g)') - get_v('에리스리톨(g)') -
                get_v('알룰로오스(g)') - get_v('당알콜(g)')).clip(lower=0)

    # 성분별 에너지 계수 적용
    calc_kcal = (
        (net_carb * 4) + (get_v('단백질(g)') * 4) + (get_v('지방(g)') * 9) +
        (get_v('식이섬유(g)') * 2) + (get_v('당알콜(g)') * 2.4)
    )

    refined_df['calculated_kcal'] = calc_kcal
    refined_df['kcal_diff_ratio'] = (abs(refined_df['에너지(kcal)'] - calc_kcal) / (refined_df['에너지(kcal)'] + 1))

    # 결과 출력
    print(f"--- 전처리 및 검증 완료 요약 ---")
    print(f"1. 입력 데이터 행 수: {len(input_df)}")
    print(f"2. 이상치(100.5g 초과) 삭제 수: {len(input_df) - len(refined_df)}")
    print(f"3. 정제된 데이터(refined_df) 행 수: {len(refined_df)}")

    return refined_df

# 실제 사용 시
final_result_df = preprocess_nutrition_data(cleaned_df)

--- 전처리 및 검증 완료 요약 ---
1. 입력 데이터 행 수: 19005
2. 이상치(100.5g 초과) 삭제 수: 0
3. 정제된 데이터(refined_df) 행 수: 19005


/tmp/ipython-input-2003251920.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_grams[col] = df[col] * unit_map[unit]
/tmp/ipython-input-2003251920.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_grams[col] = df[col] * unit_map[unit]
/tmp/ipython-input-2003251920.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame

ㄴ 에너지가 잘못 작성된 행 없음

In [11]:
final_result_df.to_csv('processed_nutrition_data.csv', index=False, encoding='utf-8-sig')
final_result_df.to_excel('processed_nutrition_data.xlsx', index=False)
print('Files processed_nutrition_data.csv and processed_nutrition_data.xlsx have been created locally.')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import shutil
import os

# Source files from the current directory
source_csv = 'processed_nutrition_data.csv'
source_excel = 'processed_nutrition_data.xlsx'

# Destination folder in Google Drive (create if it doesn't exist)
destination_folder = '/content/drive/MyDrive/Nutrition_Data'

# Create the destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Copy the files to Google Drive
shutil.copy(source_csv, destination_folder)
shutil.copy(source_excel, destination_folder)

print(f"'{source_csv}' copied to '{destination_folder}'")
print(f"'{source_excel}' copied to '{destination_folder}'")

FileNotFoundError: [Errno 2] No such file or directory: 'processed_nutrition_data.csv'